<a href="https://colab.research.google.com/github/georgeaaron7/pytorch-codes/blob/main/pytorch_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch #entire
import torch.nn as nn #all the nn modules, loss fns
import torch.optim as optim #all the optimization algos
import torch.nn.functional as F #all the functions that don't have any functions, relu, tanh, etc
from torch.utils.data import DataLoader #easier dataset management
import torchvision.datasets as datasets #pytorch std datasets
import torchvision.transforms as transforms #transformations that we can perform on our datasets

In [3]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes): #input size = 28x28 mnist dataset
    super(NN, self).__init__() #super calls the initialization method of the parent class
    self.fc1 = nn.Linear(input_size, 50) #50 layers as output
    self.fc2 = nn.Linear(50, num_classes) #50 layers input from previous and output as 1-10 classes

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyperparams
input_size = 784
num_classes = 10
learning_rate = 0.005
batch_size = 64
num_epochs = 5

In [18]:
# training dataset and training data loader
train_dataset = datasets.MNIST(root='sample_data/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# test dataset and test data loader
test_dataset = datasets.MNIST(root='sample_data/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [19]:
model = NN(input_size=input_size, num_classes=num_classes).to(device) #initializing the actual model itself

In [20]:
criterion = nn.CrossEntropyLoss() #loss fn used for multi-class classification, -ln(x)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [21]:
for epoch in range(num_epochs): #each complete pass through the dataset is called an epoch
  for batch_idx, (data, targets) in enumerate(train_loader): #provides batches of data
    data = data.to(device=device) # contains the input data for the current batch
    targets = targets.to(device=device) #contains correct labels for input data in current batch
    data = data.reshape(data.shape[0], -1) #flattens in input

    # forward
    scores = model(data)
    loss = criterion(scores,targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent
    optimizer.step()

In [22]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
  model.train()

In [23]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 59004 / 60000 with accuracy 98.34
Got 9713 / 10000 with accuracy 97.13
